In [3]:
ticker = input('Please input the ticker of the stock: ')

In [4]:
#common parts
import requests
from bs4 import BeautifulSoup
link = 'https://finance.yahoo.com/quote/MS/financials?p=' + ticker
response = requests.get(link)
if response.status_code == 200:
    pass
else:
    print('An error occurred when accessing the page!')

In [5]:
def analysis(ticker):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    link = 'https://finance.yahoo.com/quote/' + ticker + '/analysis?p' + ticker
    response = requests.get(link)
    if response.status_code == 200:
        pass
    else:
        print('An error occurred when accessing the page!')
    result_page = BeautifulSoup(response.content,'lxml')
    df_list = []
    for table_tag in result_page.find_all('table'):
        col,data_col,data_frame = [], [], []
        for th_tag in table_tag.find_all('th'):
            col.append(th_tag.get_text())
        i = 1
        for td_tag in table_tag.find_all('td'):
            if 1 <= i <= 5:
                data_col.append(td_tag.get_text())
                i += 1
            else:
                data_frame.append(data_col)
                data_col = []
                i = 1
                data_col.append(td_tag.get_text())
                i += 1
        data_frame.append(data_col)
        df = pd.DataFrame(data_frame,columns = col)
        df.set_index(col[0],inplace=True)
        df_list.append(df)
    return df_list[0]
#     return df_list[0],df_list[1],df_list[2],df_list[3],df_list[4],df_list[5]

In [6]:
def income_statement(ticker):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    link = 'https://finance.yahoo.com/quote/' + ticker + '/financials?p=' + ticker
    response = requests.get(link)
    if response.status_code == 200:
        pass
    else:
        print('An error occurred when accessing the page!')
    result_page = BeautifulSoup(response.content,'lxml')
    label,col,data_col,data_frame = [],[],[],[]
    for tag in result_page.find_all('td')[:5]:
        col.append(tag.get_text())
    col[0] = col[0] + ' (All numbers in thousands)'
    i = 1
    for tag in result_page.find_all('td')[5:]:
        if tag.get('class') != ['Fw(b)', 'Fz(15px)', 'Pb(8px)', 'Pt(36px)']:
            if 1 <= i <= 5:
                data_col.append(tag.get_text())
                i += 1
            else:
                data_frame.append(data_col)
                data_col = []
                i = 1
                data_col.append(tag.get_text())
                i += 1
    data_frame.append(data_col)
    df = pd.DataFrame(data_frame,columns = col)
    df.set_index(col[0],inplace=True)
    return df

In [8]:
def balance_sheet(ticker):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    link = 'https://finance.yahoo.com/quote/' + ticker + '/balance-sheet?p=' + ticker
    response = requests.get(link)
    result_page = BeautifulSoup(response.content,'lxml')
    df_list = []
    col,data_col,data_frame = [], [], []
    i = 1
    end = 0
    for td_tag in result_page.find_all('td')[:5]:
        col.append(td_tag.get_text())
    col[0] = col[0] + ' (All numbers in thousands)'
    for td_tag in result_page.find_all('td')[5:]:
        if td_tag.get('class') in [['Fw(b)', 'Fz(15px)'],['C($gray)', 'Ta(end)']]:
            col.append(td_tag.get_text())
        elif td_tag.get('class') != ['Fw(b)', 'Fz(15px)', 'Pb(8px)', 'Pt(36px)']:
            if 1 <= i <= 5:
                data_col.append(td_tag.get_text())
                i += 1
            else:
                data_frame.append(data_col)
                data_col = []
                i = 1
                data_col.append(td_tag.get_text())
                i += 1
        if td_tag.get('class') == ['Fw(b)', 'Fz(s)', 'Ta(end)', 'Pb(20px)']:
            end += 1
        if end == 4:
            data_frame.append(data_col)
            df = pd.DataFrame(data_frame,columns = col)
            df.set_index(col[0],inplace=True)
            df_list.append(df)
            end = 0
            i = 1
            data_col,data_frame = [], []
    return df_list[0]

In [10]:
def holders(ticker):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    link = 'https://finance.yahoo.com/quote/' + ticker + '/holders?p' + ticker
    response = requests.get(link)
    result_page = BeautifulSoup(response.content,'lxml')
    df_list = []
    for div_tag in result_page.find_all('div'):
        if div_tag.get('class') in [['Mt(25px)', 'Ovx(a)', 'W(100%)'],['W(100%)', 'Mb(20px)']]:
            if div_tag.find('span').get_text() == 'Breakdown':
                j = 2
                col = ['Major Holders Breakdown',' ']
            else:
                j = 5
                col = []
            i = 1
            data_col,data_frame = [],[]
            for th_tag in div_tag.find_all('th'):
                col.append(th_tag.get_text())
            for td_tag in div_tag.find_all('td'):
                if 1 <= i <= j:
                    data_col.append(td_tag.get_text())
                    i += 1
                else:
                    data_frame.append(data_col)
                    data_col = []
                    i = 1
                    data_col.append(td_tag.get_text())
                    i += 1
            data_frame.append(data_col)
            df = pd.DataFrame(data_frame,columns = col)
            df.set_index(col[0],inplace=True)
            df_list.append(df)
    return df_list[0]
    #     print(tag.get('class'))

In [11]:
holders(ticker)

,
Major Holders Breakdown,
24.58%,% of Shares Held by All Insider
62.77%,% of Shares Held by Institutions
83.23%,% of Float Held by Institutions
"1,409",Number of Institutions Holding Shares
